<div style="text-align: right">Author: Yuktha Bhadane (yb294@cornell.edu)</div>

### Table 1: Global summary statistics Forward Analytics power sector data
This notebook analyzes the Forward Analytics Forward Analytics with global coverage of power plants at the asset level and creates a comprehensive summary table only for the **79797 operational power plants**. 

0. Set Up

In [1]:
import pandas as pd

# Load the Forward Analytics (cleaned) dataset
# Update the path as needed
df = pd.read_csv('/Users/yukthabhadane/Documents/Climate Finance Thesis/Data/v5_power_Forward_Analytics2024.csv')

# Define fuel groups
fossil_fuels = ['Coal', 'Oil', 'Gas']
renewables = ['Solar', 'Wind', 'Hydropower', 'Geothermal ']  # Note space after Geothermal
others = ['Nuclear', 'Bioenergy', 'Other']

# Filter operating plants
operating_df = df[df['status'] == 'operating']

# Group for the table
renewables_nuclear = renewables + ['Nuclear']
fossil_fuels_other = fossil_fuels + ['Other']
all_categories = fossil_fuels + renewables + others

1. Calculations

In [2]:
# Create empty dictionaries to store data
plant_counts = {}
capacity = {}
used_capacity = {}
electricity_gen = {}
emissions = {}

# Calculate metrics for each category
for category in all_categories:
    category_df = operating_df[operating_df['subsector'] == category]
    plant_counts[category] = len(category_df)
    capacity[category] = category_df['capacity'].sum() / 1000  # MW to GW
    used_capacity[category] = category_df['used_capacity'].sum() / 1000  # MW to GW
    electricity_gen[category] = used_capacity[category] * 8760 / 1000  # GW to TWh
    emissions[category] = category_df['annualco2tyear'].sum() / 1000  # million tonnes to gigatonnes

# Calculate totals
total_count = sum(plant_counts.values())
total_capacity = sum(capacity.values())
total_used = sum(used_capacity.values())
total_gen = sum(electricity_gen.values())
total_emissions = sum(emissions.values())

# Calculate subtotals
fossil_other_count = sum(plant_counts[cat] for cat in fossil_fuels_other if cat in plant_counts)
fossil_other_capacity = sum(capacity[cat] for cat in fossil_fuels_other if cat in capacity)
fossil_other_used = sum(used_capacity[cat] for cat in fossil_fuels_other if cat in used_capacity)
fossil_other_gen = sum(electricity_gen[cat] for cat in fossil_fuels_other if cat in electricity_gen)
fossil_other_emissions = sum(emissions[cat] for cat in fossil_fuels_other if cat in emissions)

renewable_nuclear_count = sum(plant_counts[cat] for cat in renewables_nuclear if cat in plant_counts)
renewable_nuclear_capacity = sum(capacity[cat] for cat in renewables_nuclear if cat in capacity)
renewable_nuclear_used = sum(used_capacity[cat] for cat in renewables_nuclear if cat in used_capacity)
renewable_nuclear_gen = sum(electricity_gen[cat] for cat in renewables_nuclear if cat in electricity_gen)
renewable_nuclear_emissions = 0  # Assuming renewables and nuclear have zero emissions

# Constants for global context (Gt)
total_energy_co2 = 37.4  # IEA (2024), CO2 Emissions in 2023, IEA, Paris https://www.iea.org/reports/co2-emissions-in-2023
total_co2 = 41.6  # #Global Carbon Budget 2024, Earth Syst. Sci. Data Discuss. [preprint], https://doi.org/10.5194/essd-2024-519, in review, 2024.

# Create the table data
data = []

# Add rows for each category
for category in all_categories:
    if category in plant_counts:
        data.append({
            'Category': category,
            'Number': plant_counts[category],
            '%': round(plant_counts[category] / total_count * 100, 1),
            'Capacity (GW)': round(capacity[category], 1),
            '% of capacity': round(capacity[category] / total_capacity * 100, 1),
            'Used Capacity (GW)': round(used_capacity[category], 1),
            '% of used capacity': round(used_capacity[category] / total_used * 100, 1),
            'Electricity Generation (TWh)': round(electricity_gen[category], 1),
            '% of generation': round(electricity_gen[category] / total_gen * 100, 1),
            'Emissions (GtCO2e)': round(emissions[category], 2),
            '% of emissions': round(emissions[category] / total_emissions * 100, 1) if total_emissions > 0 else 0,
            '% of energy related CO2 (37.4)': round(emissions[category] / total_energy_co2 * 100, 1),
            '% of total CO2 (41.6)': round(emissions[category] / total_co2 * 100, 1)
        })

# Add subtotal row for fossil fuels + other
data.append({
    'Category': 'Fossil fuels (+other)',
    'Number': fossil_other_count,
    '%': round(fossil_other_count / total_count * 100, 1),
    'Capacity (GW)': round(fossil_other_capacity, 1),
    '% of capacity': round(fossil_other_capacity / total_capacity * 100, 1),
    'Used Capacity (GW)': round(fossil_other_used, 1),
    '% of used capacity': round(fossil_other_used / total_used * 100, 1),
    'Electricity Generation (TWh)': round(fossil_other_gen, 1),
    '% of generation': round(fossil_other_gen / total_gen * 100, 1),
    'Emissions (GtCO2e)': round(fossil_other_emissions, 2),
    '% of emissions': 100.0,
    '% of energy related CO2 (37.4)': round(fossil_other_emissions / total_energy_co2 * 100, 1),
    '% of total CO2 (41.6)': round(fossil_other_emissions / total_co2 * 100, 1)
})

# Add subtotal row for renewables + nuclear
data.append({
    'Category': 'Renewables (+nuclear)',
    'Number': renewable_nuclear_count,
    '%': round(renewable_nuclear_count / total_count * 100, 1),
    'Capacity (GW)': round(renewable_nuclear_capacity, 1),
    '% of capacity': round(renewable_nuclear_capacity / total_capacity * 100, 1),
    'Used Capacity (GW)': round(renewable_nuclear_used, 1),
    '% of used capacity': round(renewable_nuclear_used / total_used * 100, 1),
    'Electricity Generation (TWh)': round(renewable_nuclear_gen, 1),
    '% of generation': round(renewable_nuclear_gen / total_gen * 100, 1),
    'Emissions (GtCO2e)': 0.0,
    '% of emissions': 0.0,
    '% of energy related CO2 (37.4)': 0.0,
    '% of total CO2 (41.6)': 0.0
})

# Add total row
data.append({
    'Category': 'Total',
    'Number': total_count,
    '%': 100.0,
    'Capacity (GW)': round(total_capacity, 1),
    '% of capacity': 100.0,
    'Used Capacity (GW)': round(total_used, 1),
    '% of used capacity': 100.0,
    'Electricity Generation (TWh)': round(total_gen, 1),
    '% of generation': 100.0,
    'Emissions (GtCO2e)': round(total_emissions, 2),
    '% of emissions': 100.0,
    '% of energy related CO2 (37.4)': round(total_emissions / total_energy_co2 * 100, 1),
    '% of total CO2 (41.6)': round(total_emissions / total_co2 * 100, 1)
})

2. Formatting the Table

In [3]:
# Create the DataFrame
summary_df = pd.DataFrame(data)

# Define the order of categories
category_order = ['Coal', 'Gas', 'Oil', 'Other', 'Solar', 'Wind', 'Hydropower', 'Geothermal ', 
                 'Bioenergy', 'Nuclear', 'Fossil fuels (+other)', 'Renewables (+nuclear)', 'Total']

# Sort the DataFrame and set index
summary_df['Category'] = pd.Categorical(summary_df['Category'], categories=category_order, ordered=True)
summary_df = summary_df.sort_values('Category')
summary_df.set_index('Category', inplace=True)

# Display the table
print("\nTable 1: Global summary statistics Forward Analytics power sector data.")
print(summary_df.to_string())

# Save the table to CSV
summary_df.to_csv('table1.csv')


Table 1: Global summary statistics Forward Analytics power sector data.
                       Number      %  Capacity (GW)  % of capacity  Used Capacity (GW)  % of used capacity  Electricity Generation (TWh)  % of generation  Emissions (GtCO2e)  % of emissions  % of energy related CO2 (37.4)  % of total CO2 (41.6)
Category                                                                                                                                                                                                                                            
Coal                     6823    8.6         2281.3           24.7              1138.8                29.5                        9976.0             29.5               10.08            71.2                            27.0                   24.2
Gas                      8043   10.1         1931.8           20.9               884.8                22.9                        7751.0             22.9                3.41            24.1   